In [8]:
import os
import sys
module_path = os.path.abspath(os.path.join('..'))
if module_path not in sys.path:
    sys.path.append(module_path)
import time
import cv2

from pySINDy.sindypde import SINDyPDE
from pySINDy import SINDy
from pySINDy.sindybase import SINDyBase
from pySINDy.sindylibr import SINDyLibr
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

def plotUV(Un, Vn, du_dt, dv_dt,x):

    ##--SAMPLING--##
    s=[78,56]
    U_ds=np.zeros(s)
    V_ds=np.zeros(s)
    d_u=np.zeros(s)
    d_v=np.zeros(s)
    for i in range(1,s[0]):
        for j in range(1,s[1]):
            U_ds[i,j]=Un[10*i,10*j]
            V_ds[i,j]=Vn[10*i,10*j]
            d_u[i,j]=np.reshape(du_dt,file_dim)[10*i,10*j]
            d_v[i,j]=np.reshape(dv_dt,file_dim)[10*i,10*j]

    fig, ax = plt.subplots(figsize=(s[1],s[0]))

    x_pos = np.arange(0,s[1],1)
    y_pos = np.arange(0,s[0],1)


    ax.quiver(x_pos,y_pos, U_ds[:,:], V_ds[:,:], width=0.001)
    ax.set_title('Plotting motion vectors')
    #plt.show()
    U_ds+=10*d_u
    V_ds+=10*d_v
    fig, ax = plt.subplots(figsize=(s[1],s[0]))

    x_pos = np.arange(0,s[1],1)
    y_pos = np.arange(0,s[0],1)
    ax.quiver(x_pos,y_pos, U_ds[:,:], V_ds[:,:], width=0.0005)
    ax.set_title('Plotting motion vectors')
    filename="%s/%i_%i_Im%i.png"%(savepath,pred_date,pred_times[x],x)
    plt.savefig(filename)
    plt.close()

def trainSINDy(U,V,dx,dy,dt):

    
    model = SINDyPDE(name='SINDyPDE model for Reaction-Diffusion Eqn')
    
    start_train=time.time()
    #model.fit(self, data, poly_degree=2, cut_off=1e-3)
    model.fit({'u': U[:,:,-1*train_size:], 'v': V[:,:,-1*train_size:]}, dt, [dx, dy], space_deriv_order=2, poly_degree=2, sample_rate=0.01, cut_off=0.01, deriv_acc=2)

    print("\n--- Train time %s seconds ---\n" %(time.time() - start_train))

    #print("\n--- Active terms ---\n" )
    size=np.shape(model.coefficients)
    cnt=0
    for i in range(size[0]):
        for j in range(size[1]):
            if (model.coefficients[i,j])!=0:
                #print(model.coefficients[i,j],"--",model.descriptions[i])
                cnt+=1
    print("--- Active terms %s ---\n" %cnt)
    return U, V, model.coefficients

def testSINDy(U,V,dx,dy,dt,coeff,x):

    
    model2 = SINDyLibr(name='Derived module from sindybase.py for libr computation')
    libx=model2.libr({'u': U[:,:,-1*train_size:], 'v': V[:,:,-1*train_size:]}, dt, [dx,dy], space_deriv_order=2, poly_degree=2, sample_rate=0.01, cut_off=0.01, deriv_acc=2)

    duv_dt=np.matmul(libx,coeff)

    du_dt=duv_dt[:,0]
    dv_dt=duv_dt[:,1]

    U_nxt=np.reshape(U,file_dim)+np.reshape(du_dt,file_dim)
    V_nxt=np.reshape(V,file_dim)+np.reshape(dv_dt,file_dim)
    
    Uname="%s/%i_%s_U%i.csv"%(savepath,pred_date,pred_times[x],x)
    Vname="%s/%i_%s_V%i.csv"%(savepath,pred_date,pred_times[x],x)
    np.savetxt(Uname,np.reshape(U,file_dim),delimiter=',')
    np.savetxt(Vname,np.reshape(V,file_dim),delimiter=',')
    
    #plotUV(U_nxt,V_nxt,du_dt,dv_dt,x)

    return U_nxt, V_nxt

def loadUV():
    start_load = time.time()
    U_mat=np.zeros(file_dim)
    V_mat=np.zeros(file_dim)
    print("\n--- Loading UV-Data.. ---\n")
    for i in range (len(inp_times)-1):
            #print("%i_%i00_%i_%i00_u.csv"%(pred_date,pred_times[i],pred_date,pred_times[i+1]))
            #print("%i_%i00_%i_%i00_v.csv"%(pred_date,pred_times[i],pred_date,pred_times[i+1]))
            U_mat = np.dstack([U_mat,pd.read_csv("Datasets/%i/%i_%s00_%i_%s00_u.csv"%(pred_date,pred_date,inp_times[i],pred_date,inp_times[i+1]),sep=',',header=None).values])
            V_mat = np.dstack([V_mat,pd.read_csv("Datasets/%i/%i_%s00_%i_%s00_v.csv"%(pred_date,pred_date,inp_times[i],pred_date,inp_times[i+1]),sep=',',header=None).values])
            
    print("\n--- Load complete.. Time: %s seconds ---\n" %(time.time() - start_load))

    return U_mat[:,:,-1*train_size:], V_mat[:,:,-1*train_size:]

def gettimes(pred_time):

    times=[['0000','0100','0200','0300','0400','0500','0600','0700','0800','0900','1000','1100','1200','1300','1400','1500','1600','1700','1800','1900','2000','2100','2200','2300'],
    ['0010','0110','0210','0310','0410','0510','0610','0710','0810','0910','1010','1110','1210','1310','1410','1510','1610','1710','1810','1910','2010','2110','2210','2310'],
    ['0020','0120','0220','0320','0420','0520','0620','0720','0820','0920','1020','1120','1220','1320','1420','1520','1620','1720','1820','1920','2020','2120','2220','2320'],
    ['0030','0130','0230','0330','0430','0530','0630','0730','0830','0930','1030','1130','1230','1330','1430','1530','1630','1730','1830','1930','2030','2130','2230','2330'],
    ['0040','0140','0240','0340','0440','0540','0640','0740','0840','0940','1040','1140','1240','1340','1440','1540','1640','1740','1840','1940','2040','2140','2240','2340'],
    ['0050','0150','0250','0350','0450','0550','0650','0750','0850','0950','1050','1150','1250','1350','1450','1550','1650','1750','1850','1950','2050','2150','2250','2350']]

    times=np.array(times)

    for i in range(24):
        #print(times[5,i-1])
        if pred_time==times[5,i-1]:
            #pred_times=times[:,i]
            pred_times=np.hstack([times[:,i],times[:,i+1],times[:,i+2]]) #next 18 frames
            #print("PredTimes:",pred_times)
            inp_times=np.hstack([times[:,i-3],times[:,i-2],times[:,i-1]]) #prev $train_size frames
            #print("InpTimes:",inp_times[-1*(train_size+1):])
            inp_times=inp_times[-1*(train_size+1):]
    return inp_times, pred_times


In [9]:


pred_date = 20190908
pred_time = '1750'
file_dim = [781,561]
#train_size = 10
start_prog=time.time() #tic

for i in range(15):
    train_size=i+3
    inp_times, pred_times = gettimes(pred_time)
    savepath = "Optimize/%i/%s-%s/%s-frames"%(pred_date,pred_times[0],pred_times[len(pred_times)-1],train_size)
    os.makedirs(savepath, exist_ok=True)

    U,V = loadUV()

    #Spatio-temporal resolution
    x_step = 1
    y_step = 1
    t_step = 10.0
    print("\n--- Train Size %i ---\n" %train_size)
    for i in range(len(pred_times)):
        print("\n--- Sequence %i ---\n" %i)
        U, V, coeff = trainSINDy(U,V,x_step,y_step,t_step)
        U_nxt, V_nxt = testSINDy(U[:,:,-1:],V[:,:,-1:],x_step,y_step,t_step, coeff, i)
        U=np.dstack([U,U_nxt])
        V=np.dstack([V,V_nxt])

print("\n--- Exec time %s seconds ---\n" %(time.time() - start_prog)) #tac
print("SINDy complete. Files saved to %s" %(savepath))


--- Loading UV-Data.. ---


--- Load complete.. Time: 0.7506270408630371 seconds ---


--- Train Size 3 ---


--- Sequence 0 ---

Dict: (1314423, 66)

--- Train time 2.8840582370758057 seconds ---

--- Active terms 6 ---

Dict: (438141, 66)

--- Sequence 1 ---

Dict: (1314423, 66)

--- Train time 2.547241687774658 seconds ---

--- Active terms 6 ---

Dict: (438141, 66)

--- Sequence 2 ---

Dict: (1314423, 66)

--- Train time 1.9872584342956543 seconds ---

--- Active terms 6 ---

Dict: (438141, 66)

--- Sequence 3 ---

Dict: (1314423, 66)

--- Train time 1.9221391677856445 seconds ---

--- Active terms 5 ---

Dict: (438141, 66)

--- Sequence 4 ---

Dict: (1314423, 66)

--- Train time 2.0978329181671143 seconds ---

--- Active terms 6 ---

Dict: (438141, 66)

--- Sequence 5 ---

Dict: (1314423, 66)

--- Train time 1.920422077178955 seconds ---

--- Active terms 5 ---

Dict: (438141, 66)

--- Sequence 6 ---

Dict: (1314423, 66)

--- Train time 5.3234922885894775 seconds ---

--- Active 

Dict: (438141, 66)

--- Sequence 5 ---

Dict: (2628846, 66)

--- Train time 4.4301042556762695 seconds ---

--- Active terms 6 ---

Dict: (438141, 66)

--- Sequence 6 ---

Dict: (2628846, 66)

--- Train time 10.719511032104492 seconds ---

--- Active terms 36 ---

Dict: (438141, 66)

--- Sequence 7 ---

Dict: (2628846, 66)

--- Train time 5.520574569702148 seconds ---

--- Active terms 24 ---

Dict: (438141, 66)

--- Sequence 8 ---

Dict: (2628846, 66)

--- Train time 5.687591791152954 seconds ---

--- Active terms 24 ---

Dict: (438141, 66)

--- Sequence 9 ---

Dict: (2628846, 66)

--- Train time 8.978368759155273 seconds ---

--- Active terms 1 ---

Dict: (438141, 66)

--- Sequence 10 ---

Dict: (2628846, 66)

--- Train time 8.460817337036133 seconds ---

--- Active terms 5 ---

Dict: (438141, 66)

--- Sequence 11 ---

Dict: (2628846, 66)

--- Train time 8.982779502868652 seconds ---

--- Active terms 27 ---

Dict: (438141, 66)

--- Sequence 12 ---

Dict: (2628846, 66)

--- Train tim

Dict: (3943269, 66)

--- Train time 16.314937114715576 seconds ---

--- Active terms 35 ---

Dict: (438141, 66)

--- Sequence 11 ---

Dict: (3943269, 66)

--- Train time 15.576191186904907 seconds ---

--- Active terms 24 ---

Dict: (438141, 66)

--- Sequence 12 ---

Dict: (3943269, 66)

--- Train time 7.610487699508667 seconds ---

--- Active terms 10 ---

Dict: (438141, 66)

--- Sequence 13 ---

Dict: (3943269, 66)

--- Train time 15.69448447227478 seconds ---

--- Active terms 0 ---

Dict: (438141, 66)

--- Sequence 14 ---

Dict: (3943269, 66)

--- Train time 16.47007393836975 seconds ---

--- Active terms 126 ---

Dict: (438141, 66)

--- Sequence 15 ---

Dict: (3943269, 66)

--- Train time 15.870245933532715 seconds ---

--- Active terms 1 ---

Dict: (438141, 66)

--- Sequence 16 ---

Dict: (3943269, 66)

--- Train time 15.546236515045166 seconds ---

--- Active terms 0 ---

Dict: (438141, 66)

--- Sequence 17 ---

Dict: (3943269, 66)

--- Train time 15.90656065940857 seconds ---



Dict: (438141, 66)

--- Sequence 16 ---

Dict: (5257692, 66)

--- Train time 25.527136087417603 seconds ---

--- Active terms 101 ---

Dict: (438141, 66)

--- Sequence 17 ---

Dict: (5257692, 66)

--- Train time 16.175938367843628 seconds ---

--- Active terms 70 ---

Dict: (438141, 66)

--- Loading UV-Data.. ---


--- Load complete.. Time: 2.750523805618286 seconds ---


--- Train Size 13 ---


--- Sequence 0 ---

Dict: (5695833, 66)

--- Train time 31.494309425354004 seconds ---

--- Active terms 12 ---

Dict: (438141, 66)

--- Sequence 1 ---

Dict: (5695833, 66)

--- Train time 31.287962436676025 seconds ---

--- Active terms 76 ---

Dict: (438141, 66)

--- Sequence 2 ---

Dict: (5695833, 66)

--- Train time 30.24722719192505 seconds ---

--- Active terms 73 ---

Dict: (438141, 66)

--- Sequence 3 ---

Dict: (5695833, 66)

--- Train time 9.792762517929077 seconds ---

--- Active terms 0 ---

Dict: (438141, 66)

--- Sequence 4 ---

Dict: (5695833, 66)

--- Train time 9.90578603744506

Dict: (438141, 66)

--- Sequence 3 ---

Dict: (7010256, 66)

--- Train time 47.96399736404419 seconds ---

--- Active terms 63 ---

Dict: (438141, 66)

--- Sequence 4 ---

Dict: (7010256, 66)

--- Train time 18.467193603515625 seconds ---

--- Active terms 1 ---

Dict: (438141, 66)

--- Sequence 5 ---

Dict: (7010256, 66)

--- Train time 18.812335729599 seconds ---

--- Active terms 0 ---

Dict: (438141, 66)

--- Sequence 6 ---

Dict: (7010256, 66)

--- Train time 19.512889623641968 seconds ---

--- Active terms 0 ---

Dict: (438141, 66)

--- Sequence 7 ---

Dict: (7010256, 66)

--- Train time 17.67105221748352 seconds ---

--- Active terms 0 ---

Dict: (438141, 66)

--- Sequence 8 ---

Dict: (7010256, 66)

--- Train time 35.66271638870239 seconds ---

--- Active terms 50 ---

Dict: (438141, 66)

--- Sequence 9 ---

Dict: (7010256, 66)

--- Train time 22.7752845287323 seconds ---

--- Active terms 4 ---

Dict: (438141, 66)

--- Sequence 10 ---

Dict: (7010256, 66)

--- Train time 44.57